In [1]:
import tkinter as tk
import tkinter.ttk as ttk
from sqlalchemy import create_engine, text
import re
import torch
from transformers import BertTokenizer, BertForMaskedLM
import os
from tkinter import messagebox

# Load pre-trained BERT model and tokenizer
model_path = "bert-translation-model-sql-json"
model = BertForMaskedLM.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Database connection parameters
DB_SERVER = "AKDEMNB"
DB_NAME = 'Bank'

p = {
    'Server': DB_SERVER,
    'Database': DB_NAME
}

if os.name == 'nt':
    driver = 'SQL Server Native Client 11.0'
else:
    driver = 'ODBC Driver 17 for SQL Server'

engine = create_engine(
    'mssql://' + p['Server'] + '/' + p['Database'] + '?driver=' + driver,
    echo=False,        
    fast_executemany=True
)

def get_user_input():
    return input_field.get()

def SQL_Query(sentence):
    date1, date2 = None, None
    amount1, amount2 = None, None
    number1, number2 = None, None

    # Regular expression pattern to match a date in YYYY-MM-DD format
    date_pattern = r"(\d{4}-\d{2}-\d{2})"
    updated_sentence = sentence

    # Search and replace dates
    date_match = re.search(date_pattern, updated_sentence)
    if date_match:
        date1 = date_match.group(1)
        updated_sentence = re.sub(date_pattern, "firstDate", updated_sentence, count=1)
        
        date_match = re.search(date_pattern, updated_sentence)
        if date_match:
            date2 = date_match.group(1)
            updated_sentence = re.sub(date_pattern, "secondDate", updated_sentence, count=1)

    # Regular expression pattern to match an integer followed by " TL"
    pattern_amount = r"(\d+) TL"
    amount_match = re.search(pattern_amount, updated_sentence)
    if amount_match:
        amount1 = int(amount_match.group(1))
        updated_sentence = re.sub(pattern_amount, "firstAmount TL", updated_sentence, count=1)

        amount_match = re.search(pattern_amount, updated_sentence)
        if amount_match:
            amount2 = int(amount_match.group(1))
            updated_sentence = re.sub(pattern_amount, "secondAmount TL", updated_sentence, count=1)

    # Regular expression pattern to match an integer
    pattern_number = r"(\d+)"
    match = re.search(pattern_number, updated_sentence)
    if match:
        number1 = int(match.group(1))
        updated_sentence = re.sub(pattern_number, "firstNumber", updated_sentence, count=1)

        match = re.search(pattern_number, updated_sentence)
        if match:
            number2 = int(match.group(1))
            updated_sentence = re.sub(pattern_number, "secondNumber", updated_sentence, count=1)

    encoded_sentence = tokenizer.encode(updated_sentence, return_tensors='pt')

    with torch.no_grad():
        outputs = model(encoded_sentence)
        predictions = torch.argmax(outputs.logits, dim=-1)

    predicted_token = tokenizer.decode(predictions[0], skip_special_tokens=True)
    predicted_token = predicted_token.replace("FROM Cheque", "FROM [Bank].[CHQ].[Cheque]")

    query = predicted_token
    if date1:
        query = query.replace("firstDate", date1)
    if date2:
        query = query.replace("secondDate", date2)
    if amount1:
        query = query.replace("firstAmount", str(amount1))
    if amount2:
        query = query.replace("secondAmount", str(amount2))
    if number1:
        query = query.replace("firstNumber", str(number1))
    if number2:
        query = query.replace("secondNumber", str(number2))

    predicted_sql_text.delete(1.0, tk.END)
    predicted_sql_text.insert(tk.END, query)

    return query

def SQL_Result(query, engine):
    query = text(query)
    try:
        with engine.connect() as connection:
            result = connection.execute(query)
            rows = result.fetchall()
            
            result_window = tk.Toplevel(root)
            result_window.title("SQL Query Result")

            table = ttk.Treeview(result_window, columns=result.keys(), show="headings", height=30)
            column_indices = {col: idx for idx, col in enumerate(result.keys())}

            for col in result.keys():
                col_index = column_indices[col]
                table.heading(col_index, text=col)

            for row in rows:
                formatted_row = [str(value) for value in row]
                table.insert("", "end", values=formatted_row)

            table.pack(pady=10)
            scrollbar = ttk.Scrollbar(result_window, orient="vertical", command=table.yview)
            scrollbar.pack(side="right", fill="y")
            table.configure(yscrollcommand=scrollbar.set)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while executing the query:\n{e}")


def execute_updated_query():
    global global_query
    updated_query = updated_sql_text.get("1.0", tk.END)
    SQL_Result(updated_query, engine)

root = tk.Tk()
root.title("Question to SQL Translator")

label = tk.Label(root, text="Enter your Turkish text")
label.pack(pady=(5,0))

input_field = tk.Entry(root, width=125)
input_field.pack(pady=(5,0))


# Add blank padding after the button
blank_padding = tk.Frame(root, height=20)
blank_padding.pack()


label = tk.Label(root, text="SQL Equivalent")
label.pack(pady=(5,0))

predicted_sql_text = tk.Text(root, height=10, width=100)
predicted_sql_text.pack(pady=(5,0))

query_button = tk.Button(root, text="Find 'SQL Result'", command=lambda: SQL_Result(SQL_Query(get_user_input()), engine))
query_button.pack(pady=(10,0))


# Add blank padding after the button
blank_padding = tk.Frame(root, height=50)
blank_padding.pack()


# Text widget for updated SQL query
updated_sql_text = tk.Text(root, height=10, width=100)
updated_sql_text.pack(pady=(5,0))

# New button for executing updated query
execute_button = tk.Button(root, text="Execute Updated Query", command=execute_updated_query)
execute_button.pack(pady=(10,0))

root.mainloop()


c:\Users\AKDEMNB\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
